In [3]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth
import numpy as np

In [5]:
retail=pd.read_excel('I:\다운로드/Online Retail.xlsx')

In [6]:
#데이터크기 확인
print(retail.shape)

(541909, 8)


In [8]:
#데이터셋 기본 정보->결측치 포함한 컬럼들 보임.
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [7]:
#결측치 확인
retail.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [9]:
#기초통계량 요약본 ->Quantiy, unitprice컬럼에 음수가 있음 -> 제거
#데이터 기간은 약 1년 
retail.describe(include='all')

C:\Temp/ipykernel_31816/1423837476.py:3: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  retail.describe(include='all')


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
count,541909.0,541909,540455,541909.000000,541909,541909.000000,406829.000000,541909
unique,25900.0,4070,4223,NaN,23260,NaN,NaN,38
top,573585.0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,NaN,2011-10-31 14:41:00,NaN,NaN,United Kingdom
freq,1114.0,2313,2369,NaN,1114,NaN,NaN,495478
first,NaN,NaN,NaN,NaN,2010-12-01 08:26:00,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,2011-12-09 12:50:00,NaN,NaN,NaN
mean,NaN,NaN,NaN,9.552250,NaN,4.611114,15287.690570,NaN
std,NaN,NaN,NaN,218.081158,NaN,96.759853,1713.600303,NaN
min,NaN,NaN,NaN,-80995.000000,NaN,-11062.060000,12346.000000,NaN
25%,NaN,NaN,NaN,1.000000,NaN,1.250000,13953.000000,NaN


In [10]:
#description이 결측치인 경우 unitprice도 0임. -> 제거 
retail[retail.Description.isnull()].UnitPrice.sum()

0.0

In [11]:
#결측치 제거
retail1=retail[retail['UnitPrice']>0]
#Custum_id도 결측치가 있지만 '누가'샀는지는 중요하지 않기에 결측치 취급 안함

In [16]:
#시간데이터 가공 ->연월일 단위까지만 고려함
retail1['ymd']=retail1['InvoiceDate'].dt.strftime('%Y%m%d')

retail1.head()

C:\Temp/ipykernel_31816/3774959598.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retail1['ymd']=retail1['InvoiceDate'].dt.strftime('%Y%m%d')


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,ymd
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,20101201
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20101201
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,20101201
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20101201
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20101201


In [15]:
re2=retail['Description'].value_counts().rename_axis('Description').reset_index(name='counts')

re2=pd.DataFrame(re2)

re2.head()

,Description,counts
0,WHITE HANGING HEART T-LIGHT HOLDER,2369
1,REGENCY CAKESTAND 3 TIER,2200
2,JUMBO BAG RED RETROSPOT,2159
3,PARTY BUNTING,1727
4,LUNCH BAG RED RETROSPOT,1638


In [17]:
#빈도가 1500개 이상인 것만 분석에 쓸 예정 -> 어느정도 팔린 것만 분석에 사용할 계획.
#연관분석에서는 한번 구매할때 구매한 양(Quantity는 중요하지 않기에 고려하지 않고, 오직 Description의 빈도만 고려)
re3=re2.loc[re2['counts']>1500]
re3
#print(re3.shape)
rank=re3.iloc[:,0]

In [18]:
#판매건수가 1500미만인 경우 분석 대상에서 제외함.
retail2=retail1.loc[retail1['Description'].isin(rank)]
print(retail2.shape)

(11584, 9)


In [19]:
#각 아이디 별, 날짜별 구매상품들 구하기. ->while문을 중첩하여 구현. 즉, 아이디별로 구매 리스트를 구하고, 그 안에서 날짜별로 구분해줌.
#단, 리스트의 길이=하루에 구매한 제품의 수가 2개 미만이면 분석에 쓸 수 없으므로 제거함.(서포트는 교집합이라 못해도 2개는 있어야 돌아감.)
a=retail2['CustomerID'].unique()
b=retail2['ymd'].unique()
i=0
j=0
list=[]
while i<len(a):
  while j<len(b):
    goods=retail2.loc[(retail2.CustomerID==a[i])&(retail2.ymd==b[j]),'Description'].unique().tolist()
    if len(goods)<2:
      break
    list.append(goods)
    j=j+1
  i=i+1

list

[['WHITE HANGING HEART T-LIGHT HOLDER', 'JUMBO BAG RED RETROSPOT'],
 ['JUMBO BAG RED RETROSPOT',
  'ASSORTED COLOUR BIRD ORNAMENT',
  'REGENCY CAKESTAND 3 TIER',
  'WHITE HANGING HEART T-LIGHT HOLDER'],
 ['LUNCH BAG RED RETROSPOT', 'WHITE HANGING HEART T-LIGHT HOLDER'],
 ['WHITE HANGING HEART T-LIGHT HOLDER', 'ASSORTED COLOUR BIRD ORNAMENT'],
 ['JUMBO BAG RED RETROSPOT', 'LUNCH BAG RED RETROSPOT'],
 ['ASSORTED COLOUR BIRD ORNAMENT', 'WHITE HANGING HEART T-LIGHT HOLDER'],
 ['PARTY BUNTING', 'ASSORTED COLOUR BIRD ORNAMENT'],
 ['LUNCH BAG RED RETROSPOT', 'ASSORTED COLOUR BIRD ORNAMENT'],
 ['WHITE HANGING HEART T-LIGHT HOLDER', 'ASSORTED COLOUR BIRD ORNAMENT'],
 ['JUMBO BAG RED RETROSPOT',
  'WHITE HANGING HEART T-LIGHT HOLDER',
  'LUNCH BAG RED RETROSPOT'],
 ['WHITE HANGING HEART T-LIGHT HOLDER', 'PARTY BUNTING'],
 ['JUMBO BAG RED RETROSPOT', 'WHITE HANGING HEART T-LIGHT HOLDER'],
 ['ASSORTED COLOUR BIRD ORNAMENT', 'REGENCY CAKESTAND 3 TIER'],
 ['JUMBO BAG RED RETROSPOT',
  'WHITE HANGING

In [20]:
te = TransactionEncoder()
te_result = te.fit(list).transform(list)

In [21]:
df=pd.DataFrame(te_result,columns=te.columns_)

In [22]:
#rank변수 값을 바꿔가며 실행해볼 수 았다.
itemset = apriori(df, min_support=0.3,use_colnames=True)
itemset

,support,itemsets
0,0.5000,(ASSORTED COLOUR BIRD ORNAMENT)
1,0.4375,(JUMBO BAG RED RETROSPOT)
2,0.3750,(LUNCH BAG RED RETROSPOT)
3,0.6875,(WHITE HANGING HEART T-LIGHT HOLDER)
4,0.3125,"(ASSORTED COLOUR BIRD ORNAMENT, WHITE HANGING ..."
5,0.3125,"(JUMBO BAG RED RETROSPOT, WHITE HANGING HEART ..."


In [26]:
#신뢰도는 지지도보다 살짝 낮게 해야 나옴.
association_rules(itemset, metric="confidence", min_threshold=0.2)
#해석=> if antecedents 사면, then consequents산다. 이다.
#둘이 동시에 거래되는 비율은 0.3정도, antecedents를 포함하는 거래 중 consequents는 비중인 신뢰도는 0.7과 0.4. 
#3,4번째가 향상도가 1이상이므로 유의미하다. (1이상일수록 연관성이 높고, consequents만 구매할 확률보다, 둘 다 구매할 확률이 높음을 의미.)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ASSORTED COLOUR BIRD ORNAMENT),(WHITE HANGING HEART T-LIGHT HOLDER),0.5000,0.6875,0.3125,0.625000,0.909091,-0.031250,0.833333
1,(WHITE HANGING HEART T-LIGHT HOLDER),(ASSORTED COLOUR BIRD ORNAMENT),0.6875,0.5000,0.3125,0.454545,0.909091,-0.031250,0.916667
2,(JUMBO BAG RED RETROSPOT),(WHITE HANGING HEART T-LIGHT HOLDER),0.4375,0.6875,0.3125,0.714286,1.038961,0.011719,1.093750
3,(WHITE HANGING HEART T-LIGHT HOLDER),(JUMBO BAG RED RETROSPOT),0.6875,0.4375,0.3125,0.454545,1.038961,0.011719,1.031250


In [25]:
#fp growth

fpgrowth(df, min_support=0.3, use_colnames=True)

,support,itemsets
0,0.6875,(WHITE HANGING HEART T-LIGHT HOLDER)
1,0.4375,(JUMBO BAG RED RETROSPOT)
2,0.5000,(ASSORTED COLOUR BIRD ORNAMENT)
3,0.3750,(LUNCH BAG RED RETROSPOT)
4,0.3125,"(JUMBO BAG RED RETROSPOT, WHITE HANGING HEART ..."
5,0.3125,"(ASSORTED COLOUR BIRD ORNAMENT, WHITE HANGING ..."
